# Prompt Engineering with Gemini

## Setting up the environment

In [4]:
!pip install -q -U google-generativeai

In [5]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

from google.colab import userdata

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [6]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Instantiating the model

In [7]:
model = genai.GenerativeModel('gemini-1.5-flash')

## Prompt Engineering Techniques

### Zero-shot

In [8]:
prompt = """Classify the sentence into neutral, negative or positive.
Text: We loved the movie we watched last night!
Sentiment:"""
response = model.generate_content(prompt)

In [9]:
to_markdown(response.text)

> Sentiment: Positive


### Few-shot

#### Without Few-Shot

In [10]:
prompt = """Classify the sentence into lpopolp, or aufhudsh following the examples provided. Don't provide any explanation.
Text: For me, it was one of the best movies I watched in my whole life."""
response = model.generate_content(prompt)
to_markdown(response.text)

> lpopolp


#### With Few-Shot

In [11]:
prompt = """Classify the sentence into lpopolp, or aufhudsh following the examples provided. Don't provide any explanation.
Text: We loved the movie we watched last night!
Sentiment: lpopolp
Text: Bro, I hated the movie!
Sentiment: aufhudsh
Text: For me, it was one of the best movies I watched in my whole life."""
response = model.generate_content(prompt)
to_markdown(response.text)

> lpopolp


### Retrieval Augmented Generation

#### Without RAG

In [12]:
prompt = """Who were the authors of pymatting library?"""
response = model.generate_content(prompt)

In [13]:
to_markdown(response.text)

> The primary author of the PyMatting library is **Raymond Yeh**.  While others may have contributed, he's credited as the main developer and driving force behind the project.


#### With RAG

In [14]:
!pip install sentence-transformers faiss-cpu -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 65.9 MB/s eta 0:00:00


In [15]:
import os

import markdown
import faiss
import torch
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

##### Downloading your documents

In [16]:
!mkdir readmes
!wget https://huggingface.co/datasets/pedrogengo/readmes/raw/main/transformer.md -O readmes/transformer.md
!wget https://huggingface.co/datasets/pedrogengo/readmes/raw/main/pymatting.md -O readmes/pymatting.md
!wget https://huggingface.co/datasets/pedrogengo/readmes/raw/main/yolo.md -O readmes/yolo.md

--2024-11-23 05:20:49--  https://huggingface.co/datasets/pedrogengo/readmes/raw/main/transformer.md
Resolving huggingface.co (huggingface.co)... 3.167.112.38, 3.167.112.25, 3.167.112.96, ...
Connecting to huggingface.co (huggingface.co)|3.167.112.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112946 (110K) [text/plain]
Saving to: ‘readmes/transformer.md’

readmes/transformer 100%[===================>] 110.30K  --.-KB/s    in 0.006s  

2024-11-23 05:20:49 (19.4 MB/s) - ‘readmes/transformer.md’ saved [112946/112946]

--2024-11-23 05:20:49--  https://huggingface.co/datasets/pedrogengo/readmes/raw/main/pymatting.md
Resolving huggingface.co (huggingface.co)... 3.167.112.38, 3.167.112.25, 3.167.112.96, ...
Connecting to huggingface.co (huggingface.co)|3.167.112.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6794 (6.6K) [text/plain]
Saving to: ‘readmes/pymatting.md’

readmes/pymatting.m 100%[===================>]   6.63K  --.-KB/s   

##### Splitting your documents

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
def md_to_text(md):
    html = markdown.markdown(md)
    soup = BeautifulSoup(html, features='html.parser')
    return soup.get_text()

def split_text(document_path, chunk_size, overlap):
  with open(document_path, 'r', encoding='utf-8') as file:
        content = file.read()

  content = md_to_text(content)
  chunks = []
  for i in range(0, len(content), chunk_size):
    chunk = document_path + content[max(0, i-overlap):min(len(content), i+chunk_size)] #pymatting.md content[0:100]. -> pymatting.md content[100:200] ...
    chunks.append(chunk)
  return chunks

In [19]:
readmes_dir = "readmes"
chunk_size = 1000
overlap = 200

all_chunks = []
for f in os.listdir(readmes_dir):
  if f[-2:] == "md":
    all_chunks += split_text(os.path.join(readmes_dir, f), chunk_size, overlap)

In [20]:
len(all_chunks)

91

##### Indexing your documents

In [21]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
_ = embedding_model.to(device)
embeddings = embedding_model.encode(all_chunks)
embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(91, 384)

In [22]:
d = embeddings.shape[1] # 384
index = faiss.IndexFlatL2(d)
index.add(embeddings)

##### Building your RAG prompt

In [23]:
prompt = """Who were the authors of pymatting library?"""
embedding = embedding_model.encode([prompt])
_, I = index.search(embedding, 5)

context = "Relevant documents:\n"
for i in I[0]:
  context += f"Doc {i+1}: {all_chunks[i]}\n"

final_prompt = f"Use the documents to answer the user.\n{context}\n{prompt}"

response = model.generate_content(final_prompt)

In [24]:
print(final_prompt)

Use the documents to answer the user.
Relevant documents:
Doc 4: readmes/pymatting.mded in this project.
Projects using PyMatting

Rembg - an excellent tool for removing image backgrounds.
PaddleSeg - a library for a wide range of image segmentation tasks.
chaiNNer - a node-based image processing GUI.
LSA-Matting - improving deep image matting via local smoothness assumption.

License
This project is licensed under the MIT License - see the LICENSE.md file for details
Citing
If you found PyMatting to be useful for your work, please consider citing our paper:
@article{Germer2020,
  doi = {10.21105/joss.02481},
  url = {https://doi.org/10.21105/joss.02481},
  year = {2020},
  publisher = {The Open Journal},
  volume = {5},
  number = {54},
  pages = {2481},
  author = {Thomas Germer and Tobias Uelwer and Stefan Conrad and Stefan Harmeling},
  title = {PyMatting: A Python Library for Alpha Matting},
  journal = {Journal of Open Source Software}
}
References
[1]
Anat Levin, Dani Lischinski

In [25]:
to_markdown(response.text)

> The authors of the PyMatting library are Thomas Germer, Tobias Uelwer, Stefan Conrad, and Stefan Harmeling.


### Chain-of-Thought

#### Without Cot

In [26]:
prompt = """Q: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?
A: The answer is 01/05/2015.
Q: The first day of 2019 is a Tuesday, and today is the first Monday of 2019. What is the date today in MM/DD/YYYY?"""
response = model.generate_content(prompt)
to_markdown(response.text) ## 01/07/2019.

> The first day of 2019 is a Tuesday.  The first Monday of 2019 would be January 7th, 2019.
> 
> So the answer is **01/07/2019**


#### With CoT

In [27]:
prompt = """Q: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?
A: If 2015 is coming in 36 hours, then it is coming in 2 days. 2 days before 01/01/2015 is 12/30/2014, so today is 12/30/2014. So one week from today will be 01/05/2015. So the answer is 01/05/2015.
Q: The first day of 2019 is a Tuesday, and today is the first Monday of 2019. What is the date today in MM/DD/YYYY?"""
response = model.generate_content(prompt)

In [28]:
to_markdown(response.text)

> The first day of 2019 is a Tuesday.  If today is the *first* Monday of 2019, that means today is January 7th, 2019.
> 
> So the answer is $\boxed{01/07/2019}$


### Self-Consistency

In [29]:
prompt = """Q: For a, b, c>0, are there possible values of them to make (a^3*b+b^3*c+c^3*a)/(a+b+c) less than abc?
A: We have (a/sqrt(c)*sqrt(c)+b/sqrt(a)*sqrt(a)+c/sqrt(b)*sqrt(b))^2≤((a/sqrt(c))^2+(b/sqrt(a))^2+(c/sqrt(b))^2)((sqrt(a))^2+(sqrt(b))^2+(sqrt(c))^2), which simplifies to (a+b+c)^2≤(a^2/c+b^2/a+c^2/b)(a+b+c). Thus, a+b+c≤(a^3*b+b^3*c+c^3*a)/(abc). So (a^3*b+b^3*c+c^3*a)/(a+b+c) is always at least abc, and there are no possible such values. So the answer is No.
Q: The product of three positive real numbers is 1, and their sum is greater than the sum of their reciprocals. How many of them can be greater than 1?"""

#### Without Self-Consistency

In [30]:
response = model.generate_content(prompt)
to_markdown(response.text)

> Let the three positive real numbers be a, b, and c.  We are given that abc = 1 and a + b + c > 1/a + 1/b + 1/c.
> 
> Since abc = 1, we can rewrite the inequality as:
> 
> a + b + c > (bc + ac + ab)/abc = bc + ac + ab
> 
> Let's rearrange the inequality:
> 
> a + b + c - ab - ac - bc > 0
> 
> Now let's consider the possible number of values greater than 1.
> 
> * **If all three are greater than 1:**  Let's say a = 2, b = 2. Then c = 1/4.  In this case, a + b + c = 4.25 and 1/a + 1/b + 1/c = 0.5 + 0.5 + 4 = 5.  The inequality a + b + c > 1/a + 1/b + 1/c is not satisfied.
> 
> * **If two are greater than 1:** Let a = 2, b = 2. Then c = 1/4.  The sum is 4.25 and the sum of reciprocals is 5.  This doesn't satisfy the condition.
> 
> * **If one is greater than 1:** Let a = 2. Then bc = 1/2.  If b = 1, c = 1/2.  Then a + b + c = 3.5 and 1/a + 1/b + 1/c = 0.5 + 1 + 2 = 3.5. This doesn't satisfy the inequality strictly.  However, if we choose a=2, b=0.6, c=1/1.2 ≈ 0.833, then a+b+c ≈ 3.433 and 1/a+1/b+1/c ≈ 0.5+1.666+1.2 ≈ 3.366.  The inequality holds true in this case.
> 
> Let's analyze further.  Suppose a > 1, b > 1. Then c < 1.  If we let a and b approach infinity, c approaches 0.  Then the sum a + b + c will be much larger than the sum of the reciprocals (1/a + 1/b + 1/c).
> 
> **Conclusion:** At most one of the three numbers can be greater than 1.  If two or more are greater than 1, then their product is greater than 1, and the third number would have to be less than 1 to make the overall product equal to 1.  This tends to make the sum of the reciprocals larger than the sum of the numbers.
> 
> Therefore, the answer is $\boxed{1}$.


#### With Self-Consistency

In [31]:
config = genai.GenerationConfig(temperature=0.5, top_k=40)

In [32]:
responses = []
for _ in range(7):
  response = model.generate_content(prompt, generation_config=config)
  responses.append(response.text)

In [33]:
for response in responses:
  print(response.split("Conclusion")[-1])
  print("-"*20)

Let the three positive real numbers be a, b, and c. We are given that abc = 1 and a + b + c > 1/a + 1/b + 1/c.

Since abc = 1, we can rewrite the inequality as:
a + b + c > a^2bc + ab^2c + abc^2
Since abc = 1, this simplifies to:
a + b + c > a + b + c
This is a contradiction.  The inequality a + b + c > 1/a + 1/b + 1/c cannot hold if abc = 1.

Let's analyze this further.  If abc = 1, then 1/a = bc, 1/b = ac, 1/c = ab.  The inequality becomes:

a + b + c > bc + ac + ab

Let's consider the case where a = 2, b = 2, and c = 1/4.  Then abc = 1.
a + b + c = 2 + 2 + 1/4 = 4.25
bc + ac + ab = (1/2) + (1/2) + 4 = 5

In this case, a + b + c < bc + ac + ab, contradicting the given condition.

Let's assume at least one of a, b, c is greater than 1.  Without loss of generality, let a > 1.  Since abc = 1, then bc < 1.  If a > 1, then at least one of b or c must be less than 1.  It's possible that both b and c are less than 1.

However, the condition a + b + c > 1/a + 1/b + 1/c is crucial.  This cond